# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7648a96c10>
├── 'a' --> tensor([[-0.6252, -0.2727,  0.3005],
│                   [ 0.0538, -0.6707,  1.2123]])
└── 'x' --> <FastTreeValue 0x7f7648a96a00>
    └── 'c' --> tensor([[-0.4248, -0.0462,  1.1552, -0.5352],
                        [ 0.5208,  0.1273, -0.5352, -0.2052],
                        [ 1.0770,  1.8257,  1.3237,  0.1908]])

In [4]:
t.a

tensor([[-0.6252, -0.2727,  0.3005],
        [ 0.0538, -0.6707,  1.2123]])

In [5]:
%timeit t.a

73.1 ns ± 1.32 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7648a96c10>
├── 'a' --> tensor([[-0.4612, -0.0094, -0.9481],
│                   [ 0.9856,  0.1824, -0.3124]])
└── 'x' --> <FastTreeValue 0x7f7648a96a00>
    └── 'c' --> tensor([[-0.4248, -0.0462,  1.1552, -0.5352],
                        [ 0.5208,  0.1273, -0.5352, -0.2052],
                        [ 1.0770,  1.8257,  1.3237,  0.1908]])

In [7]:
%timeit t.a = new_value

73.1 ns ± 1.18 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6252, -0.2727,  0.3005],
               [ 0.0538, -0.6707,  1.2123]]),
    x: Batch(
           c: tensor([[-0.4248, -0.0462,  1.1552, -0.5352],
                      [ 0.5208,  0.1273, -0.5352, -0.2052],
                      [ 1.0770,  1.8257,  1.3237,  0.1908]]),
       ),
)

In [10]:
b.a

tensor([[-0.6252, -0.2727,  0.3005],
        [ 0.0538, -0.6707,  1.2123]])

In [11]:
%timeit b.a

67.1 ns ± 1.08 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4090,  1.7419,  1.7487],
               [-0.3035,  1.7611,  0.4870]]),
    x: Batch(
           c: tensor([[-0.4248, -0.0462,  1.1552, -0.5352],
                      [ 0.5208,  0.1273, -0.5352, -0.2052],
                      [ 1.0770,  1.8257,  1.3237,  0.1908]]),
       ),
)

In [13]:
%timeit b.a = new_value

625 ns ± 10.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

983 ns ± 9.15 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.9 µs ± 170 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

292 µs ± 14.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

267 µs ± 6.25 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f759e75b7f0>
├── 'a' --> tensor([[[-0.6252, -0.2727,  0.3005],
│                    [ 0.0538, -0.6707,  1.2123]],
│           
│                   [[-0.6252, -0.2727,  0.3005],
│                    [ 0.0538, -0.6707,  1.2123]],
│           
│                   [[-0.6252, -0.2727,  0.3005],
│                    [ 0.0538, -0.6707,  1.2123]],
│           
│                   [[-0.6252, -0.2727,  0.3005],
│                    [ 0.0538, -0.6707,  1.2123]],
│           
│                   [[-0.6252, -0.2727,  0.3005],
│                    [ 0.0538, -0.6707,  1.2123]],
│           
│                   [[-0.6252, -0.2727,  0.3005],
│                    [ 0.0538, -0.6707,  1.2123]],
│           
│                   [[-0.6252, -0.2727,  0.3005],
│                    [ 0.0538, -0.6707,  1.2123]],
│           
│                   [[-0.6252, -0.2727,  0.3005],
│                    [ 0.0538, -0.6707,  1.2123]]])
└── 'x' --> <FastTreeValue 0x7f759ed8ea90>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

43.6 µs ± 881 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f759e7770a0>
├── 'a' --> tensor([[-0.6252, -0.2727,  0.3005],
│                   [ 0.0538, -0.6707,  1.2123],
│                   [-0.6252, -0.2727,  0.3005],
│                   [ 0.0538, -0.6707,  1.2123],
│                   [-0.6252, -0.2727,  0.3005],
│                   [ 0.0538, -0.6707,  1.2123],
│                   [-0.6252, -0.2727,  0.3005],
│                   [ 0.0538, -0.6707,  1.2123],
│                   [-0.6252, -0.2727,  0.3005],
│                   [ 0.0538, -0.6707,  1.2123],
│                   [-0.6252, -0.2727,  0.3005],
│                   [ 0.0538, -0.6707,  1.2123],
│                   [-0.6252, -0.2727,  0.3005],
│                   [ 0.0538, -0.6707,  1.2123],
│                   [-0.6252, -0.2727,  0.3005],
│                   [ 0.0538, -0.6707,  1.2123]])
└── 'x' --> <FastTreeValue 0x7f759e777cd0>
    └── 'c' --> tensor([[-0.4248, -0.0462,  1.1552, -0.5352],
                        [ 0.5208,  0.1273, -0.5352, -0.2052],
                 

In [23]:
%timeit t_cat(trees)

40.6 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

79.4 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.6252, -0.2727,  0.3005],
                [ 0.0538, -0.6707,  1.2123]],
       
               [[-0.6252, -0.2727,  0.3005],
                [ 0.0538, -0.6707,  1.2123]],
       
               [[-0.6252, -0.2727,  0.3005],
                [ 0.0538, -0.6707,  1.2123]],
       
               [[-0.6252, -0.2727,  0.3005],
                [ 0.0538, -0.6707,  1.2123]],
       
               [[-0.6252, -0.2727,  0.3005],
                [ 0.0538, -0.6707,  1.2123]],
       
               [[-0.6252, -0.2727,  0.3005],
                [ 0.0538, -0.6707,  1.2123]],
       
               [[-0.6252, -0.2727,  0.3005],
                [ 0.0538, -0.6707,  1.2123]],
       
               [[-0.6252, -0.2727,  0.3005],
                [ 0.0538, -0.6707,  1.2123]]]),
    x: Batch(
           c: tensor([[[-0.4248, -0.0462,  1.1552, -0.5352],
                       [ 0.5208,  0.1273, -0.5352, -0.2052],
                       [ 1.0770,  1.8257,  1.3237,  0.1908]],
         

In [26]:
%timeit Batch.stack(batches)

105 µs ± 593 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.6252, -0.2727,  0.3005],
               [ 0.0538, -0.6707,  1.2123],
               [-0.6252, -0.2727,  0.3005],
               [ 0.0538, -0.6707,  1.2123],
               [-0.6252, -0.2727,  0.3005],
               [ 0.0538, -0.6707,  1.2123],
               [-0.6252, -0.2727,  0.3005],
               [ 0.0538, -0.6707,  1.2123],
               [-0.6252, -0.2727,  0.3005],
               [ 0.0538, -0.6707,  1.2123],
               [-0.6252, -0.2727,  0.3005],
               [ 0.0538, -0.6707,  1.2123],
               [-0.6252, -0.2727,  0.3005],
               [ 0.0538, -0.6707,  1.2123],
               [-0.6252, -0.2727,  0.3005],
               [ 0.0538, -0.6707,  1.2123]]),
    x: Batch(
           c: tensor([[-0.4248, -0.0462,  1.1552, -0.5352],
                      [ 0.5208,  0.1273, -0.5352, -0.2052],
                      [ 1.0770,  1.8257,  1.3237,  0.1908],
                      [-0.4248, -0.0462,  1.1552, -0.5352],
                      [ 0.5208,  

In [28]:
%timeit Batch.cat(batches)

185 µs ± 2.97 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

602 µs ± 11.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
